# Reading the geospatial data

In [6]:
import numpy as np
import pandas as pd
import os

geo = pd.read_csv(
    "01_input/olist_geolocation_dataset.csv",
    dtype={"geolocation_zip_code_prefix": str},
)

# Gets the first three and four first digits of zip codes, we will explore this further to understand how zip codes works
geo["geolocation_zip_code_prefix_1_digits"] = geo["geolocation_zip_code_prefix"].str[
    0:1
]
geo["geolocation_zip_code_prefix_2_digits"] = geo["geolocation_zip_code_prefix"].str[
    0:2
]
geo["geolocation_zip_code_prefix_3_digits"] = geo["geolocation_zip_code_prefix"].str[
    0:3
]
geo["geolocation_zip_code_prefix_4_digits"] = geo["geolocation_zip_code_prefix"].str[
    0:4
]
geo.head(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,geolocation_zip_code_prefix_1_digits,geolocation_zip_code_prefix_2_digits,geolocation_zip_code_prefix_3_digits,geolocation_zip_code_prefix_4_digits
0,01037,-23.545621,-46.639292,sao paulo,SP,0,01,010,0103
1,01046,-23.546081,-46.644820,sao paulo,SP,0,01,010,0104
2,01046,-23.546129,-46.642951,sao paulo,SP,0,01,010,0104
